In [41]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import sqlite3
import os

class BookStoreApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sistem Pembelian Buku")
        
        # Initialize database
        self.conn = sqlite3.connect('user_data.db')
        self.cursor = self.conn.cursor()
        self.create_table()

        self.login_page()

    def create_table(self):
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS users (
                username TEXT PRIMARY KEY,
                password TEXT NOT NULL
            )
        ''')
        self.conn.commit()

    def login_page(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        # Menambahkan gambar di sebelah elemen-elemen login
        login_image = Image.open("Junior Books.png")  # Ganti dengan nama file gambar yang sesuai
        login_image = login_image.resize((200, 200))  # Sesuaikan ukuran gambar
        login_photo = ImageTk.PhotoImage(login_image)
        login_label = tk.Label(self.frame, image=login_photo)
        login_label.image = login_photo
        login_label.pack(side="left", padx=10, pady=10)

        tk.Label(self.frame, text="Login").pack(pady=10)
        tk.Label(self.frame, text="Username").pack()
        self.username_entry = tk.Entry(self.frame)
        self.username_entry.pack()
        tk.Label(self.frame, text="Password").pack()
        self.password_entry = tk.Entry(self.frame, show='*')
        self.password_entry.pack()
        tk.Button(self.frame, text="Login", command=self.authenticate).pack(pady=10)
        tk.Button(self.frame, text="Register", command=self.register_page).pack(pady=5)

    def register_page(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        tk.Label(self.frame, text="Registrasi").pack(pady=10)
        tk.Label(self.frame, text="Username").pack()
        self.reg_username_entry = tk.Entry(self.frame)
        self.reg_username_entry.pack()
        tk.Label(self.frame, text="Password").pack()
        self.reg_password_entry = tk.Entry(self.frame, show='*')
        self.reg_password_entry.pack()
        tk.Button(self.frame, text="Register", command=self.save_registration).pack(pady=10)
        tk.Button(self.frame, text="Kembali", command=self.login_page).pack(pady=5)

    def save_registration(self):
        username = self.reg_username_entry.get()
        password = self.reg_password_entry.get()
        
        if username and password:
            try:
                self.cursor.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, password))
                self.conn.commit()
                messagebox.showinfo("Registrasi", "Registrasi berhasil!")
                self.login_page()
            except sqlite3.IntegrityError:
                messagebox.showerror("Registrasi", "Username sudah terdaftar!")
        else:
            messagebox.showerror("Registrasi", "Username dan Password tidak boleh kosong!")

    def authenticate(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        
        self.cursor.execute('SELECT * FROM users WHERE username = ? AND password = ?', (username, password))
        user = self.cursor.fetchone()

        if user:
            messagebox.showinfo("Login", "Login berhasil!")
            self.main_menu()
        else:
            messagebox.showerror("Login", "Username atau Password salah atau belum terdaftar!")

    def main_menu(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        tk.Label(self.frame, text="Menu Utama").pack(pady=10)
        tk.Button(self.frame, text="Pilih Produk", command=self.select_product).pack(pady=5)
        tk.Button(self.frame, text="Keluar", command=self.root.quit).pack(pady=5)

    def select_product(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        tk.Label(self.frame, text="Pilih Produk").pack(pady=10)

        products = [
            {"name": "100 Kata Pertamaku", "image": "FOTO PRODUK PRAKTIKUM/100 Kata Pertamaku/100_kata_pertamaku-1.jpg"},
            {"name": "Aktivitas Pertamaku ABC 123", "image": "FOTO PRODUK PRAKTIKUM/Aktivitas Pertamaku ABC/ABC-1.jpg"},
            {"name": "100 Hewan Pertamaku", "image": "FOTO PRODUK PRAKTIKUM/100 Hewan Pertamaku/100_hewan-1.jpg"},
            {"name": "Aku Mau Tau Semua - Jilid 1", "image": "FOTO PRODUK PRAKTIKUM/Aku Mau Tau Semua - Jilid 1/amts_v1-1.jpg"},
            {"name": "Aku Mau Tau Semua - Jilid 2", "image": "FOTO PRODUK PRAKTIKUM/Aku Mau Tau Semua - Jilid 2/amts_v2-1.jpg"},
            {"name": "Eduscience Eksperimen Keren", "image": "FOTO PRODUK PRAKTIKUM/Eduscience Eksperimen Keren/edu_eksperimen_keren-1.jpg"},
            {"name": "Eduscience Hewan Predator", "image": "FOTO PRODUK PRAKTIKUM/Eduscience Hewan Predator/edu_hewan_predator-1.jpg"},
            {"name": "Flip & Find - Lets See Inside Animal!", "image": "FOTO PRODUK PRAKTIKUM/Flip & Find - Lets See Inside Animal!/lets_see-1.jpg"},
            {"name": "Kidzopedia Hewan Dan Tumbuhan", "image": "FOTO PRODUK PRAKTIKUM/Kidzopedia Hewan Dan Tumbuhan/kidzo_hewan_tumbuhan-1.jpg"},
            {"name": "Kidzopedia Keajaiban Tubuh Manusia", "image": "FOTO PRODUK PRAKTIKUM/Kidzopedia Keajaiban Tubuh Manusia/kidzo_tubuh-1.jpg"},
            {"name": "Kidzopedia Peristiwa Alam", "image": "FOTO PRODUK PRAKTIKUM/Kidzopedia Peristiwa Alam/kidzo_peristiwa_alam-1.jpg"},
            {"name": "Lets Explore! Around the Farm", "image": "FOTO PRODUK PRAKTIKUM/Lets Explore! Around the Farm/farm-1.jpg"},
            {"name": "Little Star 15 menit Pintar Membaca", "image": "FOTO PRODUK PRAKTIKUM/Little Star 15 menit Pintar Membaca/pintar_membaca-1.jpg"},
            {"name": "Seri Amazing Science From Quran", "image": "FOTO PRODUK PRAKTIKUM/Seri Amazing Science From Quran/amazing_science-1.jpg"},
            {"name": "Tanya Ini Tanya Itu Virus dan Bakteri", "image": "FOTO PRODUK PRAKTIKUM/Tanya Ini Tanya Itu Virus dan Bakteri/virus-1.jpg"},
        ]
        row = 0
        column = 0
        
        for product in products:
            self.add_product(product, row, column)
            column += 1
            if column == 5:
                row += 1
                column = 0

        tk.Button(self.frame, text="Kembali ke Menu Utama", command=self.main_menu).pack(pady=5)

    def add_product(self, product, row, column):
        frame = tk.Frame(self.frame)
        frame.grid(row=row, column=column, padx=5, pady=5)

        try:
            image_path = product["image"]
            image = Image.open(image_path)
            image = image.resize((100, 100))
            photo = ImageTk.PhotoImage(image)

            label = tk.Label(frame, image=photo)
            label.image = photo
            label.grid(row=0, column=0, columnspan=2)

            button = tk.Button(frame, text=f"Pilih {product['name']}", command=lambda: self.select_quantity(product))
            button.grid(row=1, column=0, columnspan=2)
            
        except Exception as e:
            messagebox.showerror("Error", f"Error loading image for {product['name']}: {e}")
            return
        

    def select_quantity(self, product):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        tk.Label(self.frame, text=f"Pilih Jumlah untuk {product['name']}").pack(pady=10)
        tk.Label(self.frame, text="Masukkan jumlah produk:").pack()
        self.quantity_entry = tk.Entry(self.frame)
        self.quantity_entry.pack()
        tk.Button(self.frame, text="Tambah ke Keranjang", command=lambda: self.add_to_cart(product)).pack(pady=10)
        tk.Button(self.frame, text="Kembali ke Pemilihan Produk", command=self.select_product).pack(pady=5)

    def add_to_cart(self, product):
        quantity = self.quantity_entry.get()
        if quantity.isdigit() and int(quantity) > 0:
            self.cart = {"product": product["name"], "quantity": int(quantity)}
            messagebox.showinfo("Keranjang", f"{quantity} {product['name']} ditambahkan ke keranjang!")
            self.checkout()
        else:
            messagebox.showerror("Keranjang", "Masukkan jumlah yang valid!")

    def checkout(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        tk.Label(self.frame, text="Checkout").pack(pady=10)
        tk.Label(self.frame, text=f"Produk: {self.cart['product']}").pack()
        tk.Label(self.frame, text=f"Jumlah: {self.cart['quantity']}").pack()
        tk.Label(self.frame, text="Metode Pembayaran:").pack()
        self.payment_method = tk.Entry(self.frame)
        self.payment_method.pack()
        tk.Button(self.frame, text="Konfirmasi Pembayaran", command=self.confirm_payment).pack(pady=10)
        tk.Button(self.frame, text="Kembali ke Menu Utama", command=self.main_menu).pack(pady=5)

    def confirm_payment(self):
        payment = self.payment_method.get()
        if payment:
            self.transaction_data = {"product": self.cart["product"], "quantity": self.cart["quantity"], "payment": payment}
            messagebox.showinfo("Pembayaran", "Pembayaran berhasil dikonfirmasi!")
            self.print_invoice()
        else:
            messagebox.showerror("Pembayaran", "Masukkan metode pembayaran!")

    def print_invoice(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root)
        self.frame.pack()

        tk.Label(self.frame, text="Invoice").pack(pady=10)
        invoice_text = f"Produk: {self.transaction_data['product']}\nJumlah: {self.transaction_data['quantity']}\nMetode Pembayaran: {self.transaction_data['payment']}"
        tk.Label(self.frame, text=invoice_text).pack()
        tk.Button(self.frame, text="Transaksi Baru", command=self.main_menu).pack(pady=10)
        tk.Button(self.frame, text="Keluar", command=self.root.quit).pack(pady=5)

    def clear_frame(self):
        for widget in self.root.winfo_children():
            widget.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = BookStoreApp(root)
    root.mainloop()



Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\ziyad01\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\ziyad01\AppData\Local\Temp\ipykernel_11508\1742442948.py", line 131, in select_product
    self.add_product(product, row, column)
  File "C:\Users\ziyad01\AppData\Local\Temp\ipykernel_11508\1742442948.py", line 141, in add_product
    frame.grid(row=row, column=column, padx=5, pady=5)
  File "c:\Users\ziyad01\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 2569, in grid_configure
    self.tk.call(
_tkinter.TclError: cannot use geometry manager grid inside .!frame3 which already has slaves managed by pack
